# Интерактивная иерархия организаций с переводом описаний на русский

Этот ноутбук создаёт интерактивный граф (HTML) на основе JSON-файлов:

- `entities_full.json` — список организаций и их атрибутов;
- `connections.json` — связи между организациями.

Особенности:

- Иерархия сверху вниз (**Top-Down**)
- Адаптивный дизайн для телефонов (**height = 100vh**)
- Автоматический перевод `description` на русский через `deep-translator`
- При клике на узел выводится справка под графом


In [ ]:
!pip install pyvis deep-translator

In [ ]:
import json
from pyvis.network import Network
import textwrap
from deep_translator import GoogleTranslator

translator = GoogleTranslator(source='auto', target='ru')

def translate_text(text):
    if not text or len(text.strip()) < 3:
        return text
    try:
        return translator.translate(text)
    except Exception:
        return text

def wrap_label(label: str, width: int = 18) -> str:
    return "\n".join(textwrap.wrap(label or "", width=width, break_long_words=False, break_on_hyphens=False))

In [3]:
from pathlib import Path

CACHE_PATH = Path("translations_cache.json")

# Загружаем кэш, если существует
if CACHE_PATH.exists():
    with open(CACHE_PATH, "r", encoding="utf-8") as f:
        translation_cache = json.load(f)
else:
    translation_cache = {}

def translate_text_cached(text):
    """Переводит описание с кэшем."""
    if not text or len(text.strip()) < 3:
        return text
    if text in translation_cache:
        return translation_cache[text]
    try:
        translated = translator.translate(text)
        translation_cache[text] = translated
        # Сохраняем обновлённый кэш
        with open(CACHE_PATH, "w", encoding="utf-8") as f:
            json.dump(translation_cache, f, ensure_ascii=False, indent=2)
        return translated
    except Exception:
        return text


In [ ]:
with open('connections.json', encoding='utf-8') as f:
    connections = json.load(f)
with open('entities_full.json', encoding='utf-8') as f:
    entities = json.load(f)

entity_map = {row['id']: row['value']['attributes'] for row in entities['rows'] if 'attributes' in row['value']}
print(f'Загружено элементов: {len(entity_map)}')

In [ ]:
net = Network(height='100vh', width='100%', bgcolor='#111111', font_color='white', directed=True)
net.set_options('''{
  "layout": {"hierarchical": {"enabled": true, "direction": "UD", "sortMethod": "hubsize", "levelSeparation": 250, "nodeSpacing": 280, "treeSpacing": 350, "shakeTowards": "roots"}},
  "interaction": {"zoomView": true, "dragView": true},
  "physics": {"enabled": false},
  "nodes": {"shape": "dot", "size": 14, "font": {"size": 18, "strokeWidth": 2, "multi": "html"}, "margin": 12},
  "edges": {"arrows": {"to": {"enabled": true, "scaleFactor": 0.6}}, "smooth": {"type": "cubicBezier"}}
}''')

In [ ]:
for eid, attrs in entity_map.items():
    label = attrs.get('label', eid)
    etype = attrs.get('element type', 'Unknown')
    parent = attrs.get('parent organization', '')
    desc = translate_text_cached(attrs.get("description", ""))
    # desc = translate_text(attrs.get('description', ''))
    website = attrs.get('website', '')
    location = attrs.get('location', '')
    color = '#4da6ff' if etype == 'Air Force' else '#80ff80' if etype == 'Navy' else '#ffcc00' if etype == 'Army' else '#ff6666'
    html_info = f"""<b>{label}</b><br>Тип: {etype}<br>Подразделение: {parent}<br>Местоположение: {location}<br>Сайт: <a href='{website}' target='_blank'>{website}</a><br><i>Описание (рус): {desc}</i>"""
    net.add_node(eid, label=wrap_label(label), title=html_info, color=color)
print('✅ Узлы добавлены с переводом описаний.')

In [ ]:
added_edges = 0
for row in connections['rows']:
    conn = row['value']
    from_id = conn.get('from_id')
    to_id = conn.get('to_id')
    if from_id in entity_map and to_id in entity_map:
        net.add_edge(from_id, to_id)
        added_edges += 1
print(f'✅ Добавлено связей: {added_edges}')

In [ ]:
custom_html = '''<div id="org_info" style="background:#111;color:white;font-size:16px;padding:10px;min-height:100px;margin-top:10px;"><b>Нажмите на узел, чтобы увидеть описание организации.</b></div><script type="text/javascript">network.on("click", function (params) {if (params.nodes.length > 0) {var nodeId = params.nodes[0];var node = nodes.get(nodeId);document.getElementById("org_info").innerHTML = node.title || "Нет описания.";}});</script>'''
net.write_html('org_hierarchy.html', notebook=False)
with open('org_hierarchy.html', 'r+', encoding='utf-8') as f:
    html = f.read()
    html = html.replace('</body>', custom_html + '\n</body>')
    f.seek(0)
    f.write(html)
    f.truncate()
print('✅ Готово! Файл org_hierarchy.html можно открыть в браузере.')

In [9]:
!pip install pyvis deep-translator -q

import json, textwrap
from pathlib import Path
from pyvis.network import Network
from deep_translator import GoogleTranslator

# === КЭШ ПЕРЕВОДОВ ===
CACHE_PATH = Path('translations_cache.json')
if CACHE_PATH.exists():
    with open(CACHE_PATH, 'r', encoding='utf-8') as f:
        translation_cache = json.load(f)
else:
    translation_cache = {}

translator = GoogleTranslator(source='auto', target='ru')

def translate_text_cached(text):
    """Перевод текста с кэшем."""
    if not text or len(text.strip()) < 3:
        return text
    if text in translation_cache:
        return translation_cache[text]
    try:
        translated = translator.translate(text)
        translation_cache[text] = translated
        with open(CACHE_PATH, 'w', encoding='utf-8') as f:
            json.dump(translation_cache, f, ensure_ascii=False, indent=2)
        return translated
    except Exception:
        return text

def wrap_label(label: str, width: int = 18) -> str:
    """Перенос строк в названиях."""
    return "\n".join(textwrap.wrap(label or "", width=width, break_long_words=False, break_on_hyphens=False))

# === ЗАГРУЗКА ДАННЫХ ===
with open('connections.json', encoding='utf-8') as f:
    connections = json.load(f)
with open('entities_full.json', encoding='utf-8') as f:
    entities = json.load(f)

entity_map = {row['id']: row['value']['attributes'] for row in entities['rows'] if 'attributes' in row['value']}
print(f"📦 Загружено элементов: {len(entity_map)}")

# === ИНВЕРТИРУЕМ ВСЕ СВЯЗИ (чтобы корни были сверху) ===
inverted_edges = []
for row in connections['rows']:
    conn = row['value']
    a, b = conn.get('from_id'), conn.get('to_id')
    if a and b:
        inverted_edges.append((b, a))  # поменяли направление

# === ФУНКЦИЯ ДЛЯ ПОСТРОЕНИЯ ГРАФА ===
def build_graph(name, subset_ids=None):
    net = Network(height='100vh', width='100%', bgcolor='#111111', font_color='white', directed=True)
    net.set_options("""
    {
      "layout": {"hierarchical": {
        "enabled": true,
        "direction": "UD",
        "sortMethod": "directed",
        "levelSeparation": 250,
        "nodeSpacing": 250,
        "treeSpacing": 300
      }},
      "interaction": {"zoomView": true, "dragView": true},
      "physics": {"enabled": false},
      "nodes": {"shape": "dot", "size": 14, "font": {"size": 18, "strokeWidth": 2, "multi": "html"}, "margin": 10},
      "edges": {"arrows": {"to": {"enabled": true, "scaleFactor": 0.6}}, "smooth": {"type": "cubicBezier"}}
    }
    """)

    for eid, attrs in entity_map.items():
        if subset_ids and eid not in subset_ids:
            continue
        label = attrs.get("label", eid)
        etype = attrs.get("element type", "Unknown")
        parent = attrs.get("parent organization", "")
        desc = translate_text_cached(attrs.get("description", ""))
        website = attrs.get("website", "")
        location = attrs.get("location", "")
        color = (
            "#4da6ff" if etype == "Air Force" else
            "#80ff80" if etype == "Navy" else
            "#ffcc00" if etype == "Army" else
            "#ff6666"
        )
        html_info = f"""
        <b>{label}</b><br>
        Тип: {etype}<br>
        Подразделение: {parent}<br>
        Местоположение: {location}<br>
        Сайт: <a href='{website}' target='_blank'>{website}</a><br>
        <i>Описание (рус): {desc}</i>
        """
        net.add_node(eid, label=wrap_label(label), title=html_info, color=color)

    # добавляем инвертированные рёбра
    for a, b in inverted_edges:
        if a in entity_map and b in entity_map:
            if subset_ids and (a not in subset_ids or b not in subset_ids):
                continue
            net.add_edge(a, b)

    file_name = f"graph_{name}.html"
    net.write_html(file_name)
    return file_name

# === СОЗДАЁМ ВСЕ ГРАФЫ ===
html_parts = []
html_parts.append(("Общая структура", build_graph("main")))

for branch in ["Air Force", "Navy", "Army"]:
    subset_ids = [eid for eid, a in entity_map.items() if a.get("element type") == branch]
    if len(subset_ids) > 5:
        html_parts.append((branch, build_graph(branch.replace(" ", "_").lower(), subset_ids)))

# === СОБИРАЕМ ВКЛАДКИ В ОДИН HTML ===
tabs_html = """
<html><head><meta charset='utf-8'><title>Иерархия организаций</title>
<style>
body{background:#111;color:white;font-family:sans-serif;}
.tab{display:inline-block;padding:10px 20px;margin:2px;background:#333;color:#fff;cursor:pointer;border-radius:6px;}
.tab.active{background:#666;}
iframe{width:100%;height:90vh;border:none;margin-top:10px;}
</style></head><body>
<h2>⚙️ Иерархия научных и военных организаций США</h2>
<div id='tabs'>
"""
for i, (name, path) in enumerate(html_parts):
    active = 'active' if i == 0 else ''
    display_name = name.replace("Air Force", "ВВС").replace("Navy", "ВМФ").replace("Army", "Сухопутные войска")
    tabs_html += f"<div class='tab {active}' onclick='openTab({i})'>{display_name}</div>"
tabs_html += "</div>"
for i, (name, path) in enumerate(html_parts):
    display = "block" if i == 0 else "none"
    tabs_html += f"<iframe id='frame{i}' src='{path}' style='display:{display}'></iframe>"
tabs_html += """
<script>
function openTab(n){
  var tabs=document.getElementsByClassName('tab');
  var frames=document.getElementsByTagName('iframe');
  for(var i=0;i<tabs.length;i++){tabs[i].classList.remove('active');frames[i].style.display='none';}
  tabs[n].classList.add('active');frames[n].style.display='block';
}
</script></body></html>
"""

with open("org_hierarchy_full.html", "w", encoding="utf-8") as f:
    f.write(tabs_html)

print("✅ Готово: org_hierarchy_full.html создан с инвертированными связями и деревом сверху вниз (Air Force / Navy / Army в вершине).")



[notice] A new release of pip is available: 25.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
📦 Загружено элементов: 359
✅ Готово: org_hierarchy_full.html создан с инвертированными связями и деревом сверху вниз (Air Force / Navy / Army в вершине).


In [22]:
!pip install pyvis deep-translator -q

import json, textwrap, re
from pathlib import Path
from collections import defaultdict, deque
from pyvis.network import Network
from deep_translator import GoogleTranslator

# === Настройки и кэш перевода ===
CACHE_PATH = Path('translations_cache.json')
if CACHE_PATH.exists():
    with open(CACHE_PATH, 'r', encoding='utf-8') as f:
        translation_cache = json.load(f)
else:
    translation_cache = {}

translator = GoogleTranslator(source='auto', target='ru')

def translate_text_cached(text):
    """Перевод с кэшем"""
    if not text or len(text.strip()) < 3:
        return text
    if text in translation_cache:
        return translation_cache[text]
    try:
        tr = translator.translate(text)
        translation_cache[text] = tr
        with open(CACHE_PATH, 'w', encoding='utf-8') as f:
            json.dump(translation_cache, f, ensure_ascii=False, indent=2)
        return tr
    except Exception:
        return text

def wrap_label(label: str, width: int = 18) -> str:
    return "\n".join(textwrap.wrap(label or "", width=width, break_long_words=False, break_on_hyphens=False))

def norm(s): return re.sub(r"\s+", " ", (s or "").strip().lower())

# === Загрузка данных ===
with open('entities_full.json', encoding='utf-8') as f:
    entities = json.load(f)
with open('connections.json', encoding='utf-8') as f:
    connections = json.load(f)

entity_map = {
    row["id"]: row["value"]["attributes"]
    for row in entities["rows"]
    if "attributes" in row["value"]
}
print(f"📦 Загружено элементов: {len(entity_map)}")

id2label = {eid: attrs.get("label", eid) for eid, attrs in entity_map.items()}

# === Инвертируем связи (родитель → ребёнок) ===
inverted_edges = []
for row in connections["rows"]:
    conn = row["value"]
    a, b = conn.get("from_id"), conn.get("to_id")
    if a and b:
        inverted_edges.append((b, a))

adj = defaultdict(list)
for u, v in inverted_edges:
    if u in entity_map and v in entity_map:
        adj[u].append(v)

# === Находим подчинённых Secretary of Defense ===
SECDEF_LABELS = {norm("Secretary of Defense"), norm("Office of the Secretary of Defense"), norm("OSD")}
top_level_ids = []
for eid, attrs in entity_map.items():
    parent = attrs.get("parent organization", "")
    if norm(parent) in SECDEF_LABELS:
        top_level_ids.append(eid)

print(f"🔝 Найдено верхних организаций: {len(top_level_ids)}")

# === Вспомогательная функция для обхода вниз ===
def collect_descendants(root_id, limit_depth=None):
    nodes = {root_id}
    edges = []
    q = deque([(root_id, 0)])
    while q:
        u, d = q.popleft()
        if limit_depth is not None and d >= limit_depth:
            continue
        for v in adj.get(u, []):
            edges.append((u, v))
            if v not in nodes:
                nodes.add(v)
                q.append((v, d + 1))
    return nodes, edges

# === Универсальный билдер графов ===
def build_graph_html(name, nodes_subset, edges_subset):
    """Создание графа с кликабельными узлами и всплывающим описанием рядом с курсором."""
    net = Network(height='90vh', width='100%', bgcolor='#111111', font_color='white', directed=True)
    net.set_options("""
    {
      "layout": {
        "hierarchical": {
          "enabled": true,
          "direction": "UD",
          "sortMethod": "directed",
          "levelSeparation": 240,
          "nodeSpacing": 260,
          "treeSpacing": 300
        },
        "improvedLayout": false,
        "randomSeed": 42
      },
      "physics": {"enabled": false},
      "interaction": {"zoomView": true, "dragView": true},
      "nodes": {
        "shape": "dot",
        "size": 14,
        "font": {"size": 18, "strokeWidth": 2, "multi": "html"},
        "margin": 12
      },
      "edges": {
        "arrows": {"to": {"enabled": true, "scaleFactor": 0.6}},
        "smooth": {"type": "cubicBezier"}
      }
    }
    """)

    node_details = {}

    for eid in nodes_subset:
        attrs = entity_map[eid]
        label = attrs.get("label", eid)
        etype = attrs.get("element type", "Unknown")
        parent = attrs.get("parent organization", "")
        desc = translate_text_cached(attrs.get("description", ""))
        location = attrs.get("location", "")
        website = attrs.get("website", "")
        color = (
            "#4da6ff" if etype == "Air Force" else
            "#80ff80" if etype == "Navy" else
            "#ffcc00" if etype == "Army" else
            "#ff6666"
        )

        net.add_node(eid, label=wrap_label(label), color=color)
        node_details[eid] = {
            "label": label,
            "etype": etype,
            "parent": parent,
            "location": location,
            "website": website,
            "desc": desc
        }

    for u, v in edges_subset:
        if u in nodes_subset and v in nodes_subset:
            net.add_edge(u, v)

    fname = f"graph_{name}.html"
    net.write_html(fname)

    # Добавляем всплывающую карточку рядом с курсором
    with open(fname, "r+", encoding="utf-8") as f:
        html = f.read()
        node_json = json.dumps(node_details, ensure_ascii=False).replace("</", "<\\/")

        extra = f"""
        <style>
        #nodePopup {{
          position: absolute;
          display: none;
          background: rgba(20,20,20,0.95);
          color: #fff;
          padding: 12px 16px;
          border-radius: 10px;
          max-width: 400px;
          box-shadow: 0 4px 10px rgba(0,0,0,0.4);
          font-size: 15px;
          line-height: 1.4;
          z-index: 9999;
        }}
        #nodePopup h3 {{
          margin-top: 0;
          color: #fff;
          font-size: 18px;
        }}
        #nodePopup a {{ color: #6cf; }}
        </style>

        <div id="nodePopup"></div>
        <script>
        const nodeData = {node_json};
        const popup = document.getElementById("nodePopup");

        network.on("click", function (params) {{
          if (params.nodes.length > 0) {{
            const id = params.nodes[0];
            const n = nodeData[id];
            if (!n) return;
            const html = `
              <h3>${{n.label}}</h3>
              <p><b>Тип:</b> ${{n.etype || '—'}}<br>
                 <b>Подразделение:</b> ${{n.parent || '—'}}<br>
                 <b>Местоположение:</b> ${{n.location || '—'}}<br>
                 <b>Сайт:</b> <a href='${{n.website || '#'}}' target='_blank'>${{n.website || '—'}}</a></p>
              <div style='margin-top:8px;color:#ccc;font-size:14px;'>${{n.desc || 'Нет описания.'}}</div>`;
            popup.innerHTML = html;
            popup.style.display = 'block';
            // позиционируем возле курсора
            const e = params.event.srcEvent;
            popup.style.left = (e.pageX + 20) + 'px';
            popup.style.top = (e.pageY + 20) + 'px';
          }} else {{
            popup.style.display = 'none';
          }}
        }});

        // скрываем при клике мимо узлов
        network.on("click", function(params) {{
          if (params.nodes.length === 0) {{
            popup.style.display = 'none';
          }}
        }});
        </script>
        """

        html = html.replace("</body>", extra + "\n</body>")
        f.seek(0)
        f.write(html)
        f.truncate()

    return fname



# === Общая вкладка (только Secretary of Defense → дети, без внуков) ===
VIRTUAL_SECDEF_ID = "virtual-secdef"
entity_map[VIRTUAL_SECDEF_ID] = {"label": "Secretary of Defense", "parent organization": ""}
general_nodes = {VIRTUAL_SECDEF_ID} | set(top_level_ids)
general_edges = [(VIRTUAL_SECDEF_ID, x) for x in top_level_ids]
general_html = build_graph_html("general", general_nodes, general_edges)

# === Строим по каждой крупной организации ===
tabs = [("Общая структура", general_html)]
misc_nodes, misc_edges = set(), []

for eid in top_level_ids:
    nodes, edges = collect_descendants(eid)
    if len(nodes) < 10:
        misc_nodes |= nodes
        misc_edges += edges
        continue
    title = id2label[eid]
    html_name = build_graph_html(f"top_{eid}", nodes, edges)
    tabs.append((title, html_name))

# === Вкладка "Прочее" (все мелкие объединены) ===
if misc_nodes:
    misc_html = build_graph_html("misc", misc_nodes, misc_edges)
    tabs.append(("Прочее", misc_html))

# === Сборка финального HTML ===
html = """
<html><head><meta charset='utf-8'><title>Иерархия организаций Минобороны США</title>
<style>
body{background:#111;color:white;font-family:sans-serif;margin:0;padding:16px;}
h2{margin:0 0 10px 0;}
#tabs{margin-top:8px;}
.tab{display:inline-block;padding:10px 16px;margin:2px;background:#333;color:#fff;cursor:pointer;border-radius:6px;font-size:15px;}
.tab.active{background:#666;}
iframe{width:100%;height:90vh;border:none;margin-top:10px;background:#1a1a1a;border-radius:8px;}
</style></head><body>
<h2>⚙️ Иерархия научных и военных организаций США</h2>
<div id='tabs'>
"""
for i, (name, path) in enumerate(tabs):
    active = 'active' if i == 0 else ''
    html += f"<div class='tab {active}' onclick='openTab({i})'>{name}</div>"
html += "</div>"
for i, (name, path) in enumerate(tabs):
    display = "block" if i == 0 else "none"
    html += f"<iframe id='frame{i}' src='{path}' style='display:{display}'></iframe>"
html += """
<script>
function openTab(n){
  var tabs=document.getElementsByClassName('tab');
  var frames=document.getElementsByTagName('iframe');
  for(var i=0;i<tabs.length;i++){tabs[i].classList.remove('active');frames[i].style.display='none';}
  tabs[n].classList.add('active');frames[n].style.display='block';
}
</script></body></html>
"""
with open("org_hierarchy_full.html", "w", encoding="utf-8") as f:
    f.write(html)

print(f"✅ Готово! org_hierarchy_full.html создан ({len(tabs)} вкладок).")



[notice] A new release of pip is available: 25.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
📦 Загружено элементов: 359
🔝 Найдено верхних организаций: 20
✅ Готово! org_hierarchy_full.html создан (8 вкладок).


In [20]:
import json
from collections import defaultdict, deque
from functools import lru_cache

# === Загрузка данных ===
with open("entities_full.json", encoding="utf-8") as f:
    entities = json.load(f)
with open("connections.json", encoding="utf-8") as f:
    connections = json.load(f)

# === Идентификаторы и имена ===
entity_attrs = {
    row["id"]: row["value"]["attributes"]
    for row in entities["rows"]
    if "attributes" in row["value"]
}
id2label = {eid: attrs.get("label", eid) for eid, attrs in entity_attrs.items()}

# === Инвертируем рёбра: parent -> child (в сырье часто child->parent) ===
children = defaultdict(set)       # set чтобы не было дублей
edges_kept = 0
missing_nodes = 0

for row in connections["rows"]:
    conn = row["value"]
    a = conn.get("from_id")   # child в сыром виде
    b = conn.get("to_id")     # parent в сыром виде
    if a in id2label and b in id2label:
        parent, child = b, a
        if child != parent:
            if child not in children[parent]:
                children[parent].add(child)
                edges_kept += 1
    else:
        missing_nodes += 1

print(f"📦 Узлов с атрибутами: {len(id2label)}")
print(f"🔁 Инвертированных рёбер сохранено: {edges_kept}")
print(f"⚠️ Рёбер, где хотя бы одного узла нет в entities_full: {missing_nodes}")

# === Подсчёт всех потомков (включая многоуровневых) с мемоизацией и защитой от циклов ===
@lru_cache(maxsize=None)
def descendants_count(node_id):
    seen = set()
    stack = list(children.get(node_id, []))
    while stack:
        u = stack.pop()
        if u in seen:
            continue
        seen.add(u)
        stack.extend(children.get(u, []))
    return len(seen)

# === Собираем все узлы, у которых >20 потомков ===
THRESHOLD = 20
big_nodes = []
for eid in id2label.keys():
    cnt = descendants_count(eid)
    if cnt > THRESHOLD:
        big_nodes.append((eid, id2label[eid], cnt))

big_nodes.sort(key=lambda x: -x[2])

print(f"\n🧮 Организаций с более чем {THRESHOLD} подчинёнными (всех уровней): {len(big_nodes)}\n")
for eid, name, cnt in big_nodes[:30]:
    print(f"{cnt:4d} | {name} ({eid})")

# Если хочешь — раскомментируй, чтобы сохранить полный список в CSV
# import csv
# with open("orgs_gt20_descendants.csv", "w", newline="", encoding="utf-8") as f:
#     w = csv.writer(f)
#     w.writerow(["id", "name", "descendants_count"])
#     w.writerows(big_nodes)
# print("\n💾 Сохранил подробный список в orgs_gt20_descendants.csv")


📦 Узлов с атрибутами: 359
🔁 Инвертированных рёбер сохранено: 438
⚠️ Рёбер, где хотя бы одного узла нет в entities_full: 0

🧮 Организаций с более чем 20 подчинёнными (всех уровней): 17

 342 | elem-3HakjeOg (elem-3HakjeOg)
 342 | elem-BFGFcNdj (elem-BFGFcNdj)
 342 | elem-NUpKzTef (elem-NUpKzTef)
 342 | elem-O372uObO (elem-O372uObO)
 342 | elem-ZQbFTW4i (elem-ZQbFTW4i)
 342 | elem-eQtxR3qG (elem-eQtxR3qG)
 342 | elem-gjvH0Rqe (elem-gjvH0Rqe)
 341 | Secretary of Defense (elem-f60zqgHN)
 106 | Navy (elem-1TB0YM9D)
  72 | Air Force (elem-mjriOCXD)
  64 | USD(R&E) (elem-bPZYPUAw)
  54 | Army (elem-alLKsSbt)
  43 | Naval Sea Systems Command (NAVSEA) (elem-hN8hYXNe)
  38 | Air Force Materiel Command (elem-0ThQmeBW)
  29 | DCTO(S&T) (elem-lGNogSqJ)
  27 | Army Futures Command (elem-i73fMCKH)
  27 | Air Force Research Laboratory (elem-kJLXLkDO)


In [21]:
from pyvis.network import Network
import json, textwrap
from collections import defaultdict, deque

# === вспомогательные функции ===
def wrap_label(label: str, width: int = 20) -> str:
    return "\n".join(textwrap.wrap(label or "", width=width, break_long_words=False, break_on_hyphens=False))

def collect_descendants(root_id, max_depth=None):
    """Собрать поддерево от root_id вниз (children)."""
    nodes, edges = set([root_id]), []
    dq = deque([(root_id, 0)])
    while dq:
        node, depth = dq.popleft()
        if max_depth is not None and depth >= max_depth:
            continue
        for ch in children.get(node, []):
            if ch not in nodes:
                nodes.add(ch)
                edges.append((node, ch))
                dq.append((ch, depth + 1))
    return nodes, edges


# === функции визуализации ===
def build_graph(name, node_ids, edge_list):
    """Строим pyvis-граф и возвращаем html-файл"""
    net = Network(height="90vh", width="100%", bgcolor="#111", font_color="white", directed=True)
    net.set_options("""
    {
      "layout": {
        "hierarchical": {
          "enabled": true,
          "direction": "UD",
          "sortMethod": "directed",
          "levelSeparation": 240,
          "nodeSpacing": 260,
          "treeSpacing": 300
        }
      },
      "physics": {"enabled": false},
      "interaction": {"zoomView": true, "dragView": true},
      "nodes": {
        "shape": "dot",
        "size": 14,
        "font": {"size": 18, "multi": "html"},
        "margin": 10
      },
      "edges": {
        "arrows": {"to": {"enabled": true, "scaleFactor": 0.6}},
        "smooth": {"type": "cubicBezier"}
      }
    }
    """)

    for n in node_ids:
        label = id2label.get(n, n)
        color = (
            "#4da6ff" if "Air" in label else
            "#80ff80" if "Navy" in label else
            "#ffcc00" if "Army" in label else
            "#ff6666"
        )
        net.add_node(n, label=wrap_label(label), color=color)
    for u, v in edge_list:
        if u in node_ids and v in node_ids:
            net.add_edge(u, v)
    html_path = f"graph_{name}.html"
    net.write_html(html_path)
    return html_path


# === общий граф: только крупные 17 организаций ===
hub_ids = [
    'elem-3HakjeOg','elem-BFGFcNdj','elem-NUpKzTef','elem-O372uObO','elem-ZQbFTW4i',
    'elem-eQtxR3qG','elem-gjvH0Rqe','elem-f60zqgHN','elem-1TB0YM9D','elem-mjriOCXD',
    'elem-bPZYPUAw','elem-alLKsSbt','elem-hN8hYXNe','elem-0ThQmeBW','elem-lGNogSqJ',
    'elem-i73fMCKH','elem-kJLXLkDO'
]

hub_edges = []
for pid, children_list in children.items():
    for ch in children_list:
        if pid in hub_ids and ch in hub_ids:
            hub_edges.append((pid, ch))

general_html = build_graph("general_hubs", hub_ids, hub_edges)

# === вкладки для каждой из 17 организаций ===
tabs = [("Общая структура (17 хабов)", general_html)]

for hid in hub_ids:
    name = id2label.get(hid, hid)
    nodes, edges = collect_descendants(hid)
    html_file = build_graph(f"hub_{hid}", nodes, edges)
    tabs.append((name, html_file))


# === собираем всё в один HTML с вкладками ===
html = """
<html><head><meta charset='utf-8'><title>Военно-научная иерархия США</title>
<style>
body{background:#111;color:white;font-family:sans-serif;margin:0;padding:16px;}
h2{margin:0 0 10px 0;}
#tabs{margin-top:8px;}
.tab{display:inline-block;padding:10px 16px;margin:2px;background:#333;color:#fff;
     cursor:pointer;border-radius:6px;font-size:15px;}
.tab.active{background:#666;}
iframe{width:100%;height:90vh;border:none;margin-top:10px;background:#1a1a1a;border-radius:8px;}
</style></head><body>
<h2>⚙️ 17 крупнейших военных организаций США и их подструктуры</h2>
<div id='tabs'>
"""
for i, (name, path) in enumerate(tabs):
    active = 'active' if i == 0 else ''
    html += f"<div class='tab {active}' onclick='openTab({i})'>{name}</div>"
html += "</div>"
for i, (name, path) in enumerate(tabs):
    display = "block" if i == 0 else "none"
    html += f"<iframe id='frame{i}' src='{path}' style='display:{display}'></iframe>"
html += """
<script>
function openTab(n){
  var tabs=document.getElementsByClassName('tab');
  var frames=document.getElementsByTagName('iframe');
  for(var i=0;i<tabs.length;i++){tabs[i].classList.remove('active');frames[i].style.display='none';}
  tabs[n].classList.add('active');frames[n].style.display='block';
}
</script></body></html>
"""

with open("org_hierarchy_17hubs.html", "w", encoding="utf-8") as f:
    f.write(html)

print("✅ Готово! Файл org_hierarchy_17hubs.html создан.")


✅ Готово! Файл org_hierarchy_17hubs.html создан.
